In [70]:
import torch
import math
import torch
from torch import empty , cat , arange
from torch . nn . functional import fold , unfold

#to delete
#from torch import nn

In [3]:
# Convolution is equivalent with Unfold + Matrix Multiplication + Fold (or view to output shape)

inp = torch.randn(1, 3, 10, 12)
w = torch.randn(2, 3, 4, 5)
inp_unf = torch.nn.functional.unfold(inp, (4, 5))
out_unf = inp_unf.transpose(1, 2).matmul(w.view(w.size(0), -1).t()).transpose(1, 2)
out = torch.nn.functional.fold(out_unf, (7, 8), (1, 1))

In [72]:
"""   
class ConvolutionTransposed(object):
    def __init__(self, channels_input, channels_output, kernel_size, stride):
        self.weight = torch.empty(channels_output, channels_input, kernel_size, kernel_size).normal_()
        
        self.channel_input = channels_input
        #print('input channels', self.channel_input)
        self.kernel_size = kernel_size
        self.stride = stride
        
        print('weight',self.weight.shape)
        
        
    def forward(self, imgs):
        #print('forward')
        _,_,H,W = imgs.shape
        H_out = (H - 1)*self.stride + self.kernel_size 
        W_out = (W - 1)*self.stride + self.kernel_size 
        
        #print('Hout Wout', H_out, W_out)
        
        self.x = imgs.permute(1, 2, 3, 0).reshape(self.channel_input, -1)
        #print('x',self.x.shape)
        self.y = (self.weight.reshape(self.channel_input, -1)).t().matmul(self.x)
        #print('y',self.y.shape)
        self.y = self.y.reshape(self.y.shape[0], -1, imgs.shape[0])
        #print('y2',self.y.shape)
        self.y = self.y.permute(2, 0, 1)
        
        #print(self.y.shape)
        self.y = fold( self.y, (H_out, W_out), kernel_size=(self.kernel_size,self.kernel_size), stride=self.stride)
        
        return self.y
"""

"   \nclass ConvolutionTransposed(object):\n    def __init__(self, channels_input, channels_output, kernel_size, stride):\n        self.weight = torch.empty(channels_output, channels_input, kernel_size, kernel_size).normal_()\n        \n        self.channel_input = channels_input\n        #print('input channels', self.channel_input)\n        self.kernel_size = kernel_size\n        self.stride = stride\n        \n        print('weight',self.weight.shape)\n        \n        \n    def forward(self, imgs):\n        #print('forward')\n        _,_,H,W = imgs.shape\n        H_out = (H - 1)*self.stride + self.kernel_size \n        W_out = (W - 1)*self.stride + self.kernel_size \n        \n        #print('Hout Wout', H_out, W_out)\n        \n        self.x = imgs.permute(1, 2, 3, 0).reshape(self.channel_input, -1)\n        #print('x',self.x.shape)\n        self.y = (self.weight.reshape(self.channel_input, -1)).t().matmul(self.x)\n        #print('y',self.y.shape)\n        self.y = self.y.reshape

In [202]:
class Sigmoid (object):
    def forward(self, input):
        self.x = 1/(1 + math.e**(-input))
        return self.x
    def backward(self, gradwrtoutput):
        self.output = (self.x*(1-self.x))*gradwrtoutput
        return self.output
    def param(self):
        return None

In [203]:
class ReLU (object):
    def forward(self, input):
        self.relu = input > 0
        self.x = self.relu*input
        return self.x
    def backward(self, gradwrtoutput):
        self.output = (self.x > 0)
        self.output = self.output *  gradwrtoutput
        return self.output
    def param(self):
        return None

In [9]:
class MSE (object):
    def forward(self, predictions, targets):
        self.predictions = predictions
        self.targets = targets
        self.mse = (((self.predictions - self.targets)**2).mean())
        return self.mse
    def backward(self, gradwrtoutput):
        self.output = 2*((self.predictions - self.targets).mean())
        self.output = self.output
        return self.output

In [91]:
class Upsample(object):
    def __init__(self, factor_size):
        self.kernel = torch.ones(factor_size,factor_size)
        #print(self.kernel.shape)
        
    def forward(self,x):
        self.b, self.channels, self.s1, self.s2 = x.shape
        self.s3, self.s4 = self.kernel.shape
        x = x.reshape(self.b, self.channels, self.s1, 1, self.s2, 1)
        self.kernel = self.kernel.reshape(1, self.s3, 1, self.s4)
        return (x * self.kernel).reshape(self.b, self.channels, self.s1 * self.s3,self.s2 * self.s4) 
    
    def backward(self,gradwrtoutput):
        dL_dS = gradwrtoutput
        dS_dX = self.kernel # [1, K, 1, K]
        
        #backward x
        dL_dS_reshape = dL_dS.reshape(self.b, self.channels, self.s1, self.s3, self.s2, self.s4) #[B, C, SI, K, SI, K]
        #print('dL_dS_reshape',dL_dS_reshape.shape)
        
        dL_dX = dL_dS_reshape /(dS_dX)
        
        #print(dL_dX.shape)
        dL_dX = dL_dX.permute(0,1,2,4,3,5)
        #print(dL_dX.shape)
        
        dL_dX_red = dL_dX[:,:,:,:,0,0]
        #print(dL_dX_red.shape)
    
        #print(dL_dX.reshape(self.b, self.channels, self.s1, self.s2).shape)
        
        return dL_dX_red
    
    def param(self):
        return None
    

In [189]:
class Convolution(object):
    def __init__(self, channels_input, channels_output, kernel_size, stride):
        
        self.weight = torch.empty(channels_output, channels_input, kernel_size, kernel_size).normal_()
        self.kernel_size = kernel_size
        self.stride = stride
        self.channels_output = channels_output
        self.channels_input = channels_input
        
        self.grad = torch.empty(channels_output, channels_input, kernel_size, kernel_size)

        #print('weight',self.weight.shape)
         
        
    def forward(self, imgs):
        
        _,_,H,W = imgs.shape
        #print('h w ',H,W)
        self.H = H
        self.W = W
        self.Hout = int((H - self.kernel_size)/self.stride + 1)
        self.Wout = int((H - self.kernel_size)/self.stride + 1)
        #print('Hout Wout', self.Hout, self.Wout)
        self.x = imgs
        #print('x', self.x.shape)
        self.x_unfolded = unfold(self.x, kernel_size = (self.kernel_size, self.kernel_size), stride = self.stride)
        #print('x unfold',self.x_unfolded.shape)
        #print('w shape', self.weight.view(self.channels_output, -1).shape)
        self.y = self.x_unfolded.transpose(1, 2).matmul(self.weight.view(self.channels_output, -1).t()).transpose(1, 2)
        #print('y',self.y.shape)
        self.y = fold(self.y, (int(self.Hout), int(self.Wout)),(1,1), stride = 1)
        #self.y = self.y.view(1,10,15,15)
        return self.y  #, self_x, self_weight
    

    def backward(self,gradwrtoutput):
        dL_dS = gradwrtoutput # [B, O, SO, SO]
        dS_dX = self.weight   # weight.shape [O, I, K, K]
        #print('dL_dS', dL_dS.shape)
        #print('dS_dX', dS_dX.shape)
        
        #define the size IxKxK
        inKerKer_size = self.channels_input*self.kernel_size*self.kernel_size
        #print('inKerKer_size',inKerKer_size)
        
        dL_dS_reshape = dL_dS.reshape(1,self.channels_output,-1) # [B, O, (SOxSO)]
        #print('dL_dS_reshape',dL_dS_reshape.shape)
        dS_dX_reshape = dS_dX.reshape(self.channels_output, -1).transpose(0,1)   # [O, (IxKxK)]^T
        #print('dS_dX_reshape',dS_dX_reshape.shape)
        
        
        #backward input
        dL_dX_reshape = dS_dX_reshape @ dL_dS_reshape # [B, (IxKxK), (SOxSO)]
        #print('dL_dX_reshape',dL_dX_reshape.shape)
        dL_dX = fold(dL_dX_reshape, kernel_size = (self.kernel_size, self.kernel_size), stride = self.stride, output_size = (self.W, self.H)) # [B, I, SI, SI]
        #print('dL_dX',dL_dX.shape)
        
        
        #backward weight
        dL_dS_reshape2 = dL_dS.reshape(self.channels_output, -1) # [O, (BxSOxSO)]
        dS_dW = self.x_unfolded # [B, (IxKxK), (SOxSO)]
        dS_dW_reshape = dS_dW.reshape(-1, inKerKer_size) # [(BxSOxSO), (IxKxK))] 
        dL_dW_reshape = dL_dS_reshape2 @ dS_dW_reshape # [O, (IxKxK)]
        dL_dW = dL_dW_reshape.view(self.channels_output,  self.channels_input, self.kernel_size, self.kernel_size) # [O, I, K, K] 
        
        self.grad = dL_dW
        #print('backward weight', self.grad.shape )
        #print('original weight', self.weight.shape )
        #backward bias
       
        
        return dL_dX  
    
    def param(self):
        #print('hello convolution')
        return self.weight, self.grad


In [191]:
class Sequential (object):
    def __init__(self, *input):
        self.layers = input
        
    def forward(self, input):
        x = input
        for lay in self.layers:
            print(lay)
            x = lay.forward(x)
        return x
    
    def backward(self, gradwrtoutput):
        out = gradwrtoutput
        for lay in reversed(self.layers):
            print(lay)
            out = lay.backward(out)
        return out

In [205]:
class SGD (object):
    def __init__(self,layers, lr):
        self.lr = lr
        self.layers = layers
        
    def zero_grad(self):
        print('zero grad')
        #print(self.layers.layers)
        
        for lay in self.layers.layers:
            print(lay)
            values = lay.param()
            if lay.param() is not None:
                lay.grad = lay.grad.zero_()
          
    
    def step(self):
        
        for lay in self.layers.layers:
            print(lay)
            values = lay.param()
            if lay.param() is not None:
                #print('step', lay.weight)
                #print('step lr', self.lr*lay.grad )
                lay.weight = lay.weight + self.lr*lay.grad
                #print('step', lay.weight)

In [206]:
conv1 = Convolution(3, 3, kernel_size = 2, stride = 2) # 1,3,16,16
upsample = Upsample(2) # 1,3, 32, 32

model = Sequential(Convolution(3, 3, kernel_size = 2, stride = 2),  # 1,3,16,16
                   ReLU(),
                   Upsample(2),
                   Sigmoid()) # 1,3, 32, 32

criterion = MSE()
sigmoid = Sigmoid()
optimizer = SGD(model, 0.005)


img = torch.randn(1, 3, 32, 32)
img_error = torch.randn(1, 3, 32, 32)


x = model.forward(img)

print('x', x.shape)

img = sigmoid.forward(x)


loss = criterion.forward(img, img_error)
print('loss', loss.shape)

optimizer.zero_grad()
output = model.backward(img_error)

print('out img', out_img.shape)

optimizer.step()




x torch.Size([1, 3, 32, 32])
loss torch.Size([])
zero grad
out img torch.Size([1, 3, 32, 32])


In [160]:
criterion = MSE()

autoencoder = Sequential(Convolution(3, 3, kernel_size = 2, stride = 2),
                                    ReLU(),
                                    Upsample(2),
                                    Sigmoid())


img = torch.randn(1, 3, 32, 32)
img_error = torch.randn(1, 3, 32, 32)

predict_img = model.forward(img)
print('prediction', predict_img.shape)
loss = criterion.forward(predict_img, img_error)

backward_img = model.backward(img)

print('backward', backward_img.shape)

AttributeError: 'int' object has no attribute 'view'

In [56]:
class Model(object):
    def __init__(self):
        self.sequence = Sequential(Convolution(3, 3, kernel_size = 2, stride = 2),
                                    ReLU(),
                                    Upsample(2),
                                    Sigmoid())

        self.criterion = MSE()
        print('Model class initialisation')

    def train(self):
    
        train_input = torch.randn(1, 3, 32, 32)
        train_target = torch.randn(1, 3, 32, 32)
    
        predict_img = self.sequence.forward(train_input)
        loss = self.criterion.forward(predict_img, train_target)

        backward_img = self.sequence.backward(img)

        return backward_img

In [71]:
img = torch.randn(1, 3, 32, 32)
img_error = torch.randn(1, 3, 32, 32)

model = Model()

output = model.train()

print(output.shape)



Model class initialisation
torch.Size([1, 3, 32, 32])


In [59]:
class Autoencoder(object):
    def __init__(self):
        self.a = 10
    def train(self):
        self.a = self.a +8
        return self.a

In [61]:
model = Autoencoder()
result = model.train()

print(result)

18
